In [1]:
from pathlib import Path

In [22]:
data = Path("day16.txt").read_text()
bits = "".join(f"{int(c, base=16):04b}" for c in data)

In [24]:
def bits2int(bits):
    return int(bits, base=2)

def product(values):
    result = 1
    for v in values:
        result *= v
    return result

## Part 1

In [32]:
def parse(bits):
    offset = 0
    
    # parse the version/operator header
    version = bits2int(bits[offset:][:3])
    offset += 3
    operator = bits2int(bits[offset:][:3])
    offset += 3
    
    if operator == 4:
        # literal packet, skip over it
        done = False
        while not done:
            done = int(bits[offset]) == 0
            offset += 5
    else:
        # operator packet, parse the child packets
        length_type = int(bits[offset])
        offset += 1

        if length_type == 0:
            # length-style children, parse each child packet until the end
            length = bits2int(bits[offset:][:15])
            offset += 15
            while length > 0:
                child_version, child_length = parse(bits[offset:][:length])
                version += child_version
                offset += child_length
                length -= child_length
        else:
            # count-style children, parse N child packets
            count = bits2int(bits[offset:][:11])
            offset += 11
            for _ in range(count):
                child_version, child_length = parse(bits[offset:])
                version += child_version
                offset += child_length
    
    return version, offset


parse(bits)[0]

955

## Part 2

In [33]:
def calculate(bits):
    offset = 0
    
    # parse the version/operator headers
    _version = bits2int(bits[offset:][:3])
    offset += 3
    operator = bits2int(bits[offset:][:3])
    offset += 3
    
    if operator == 4:
        # literal packet, decode it directly
        result = 0
        done = False
        while not done:
            done = int(bits[offset]) == 0
            result = (result << 4) | bits2int(bits[offset + 1:][:4])
            offset += 5
    else:
        # operator packet, parse child packets
        params = []
        length_type = int(bits[offset])
        offset += 1
        if length_type == 0:
            # length-style children, parse each child packet until the end
            length = bits2int(bits[offset:][:15])
            offset += 15
            while length > 0:
                child_result, child_length = calculate(bits[offset:][:length])
                params.append(child_result)
                offset += child_length
                length -= child_length
        else:
            # count-style children, parse N child packets
            count = bits2int(bits[offset:][:11])
            offset += 11
            for _ in range(count):
                child_result, child_length = calculate(bits[offset:])
                params.append(child_result)
                offset += child_length
        
        # dispatch the operator
        match operator:
            case 0:
                result = sum(params)
            case 1:
                result = product(params)
            case 2:
                result = min(params)
            case 3:
                result = max(params)
            case 5:
                result = int(params[0] > params[1])
            case 6:
                result = int(params[0] < params[1])
            case 7:
                result = int(params[0] == params[1])
    
    return result, offset

calculate(bits)[0]

158135423448